In [1]:
#importing required libraries
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
import re
import os
from time import sleep 

In [2]:
def fetch_stock_info(stock_code):   #function to get the prices for respective stock
    #nse url where data gets updated regularly
    stock_url  = 'https://www1.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stock_code) 
    #my user agent
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    #establishing connection to the url
    r= requests.get(stock_url, headers=headers)
    #using beautifulsoup to parse get the html data of the website
    soup = BeautifulSoup(r.text, 'html.parser')
    #filtering html data to get the desired form
    data_array = soup.find(id='responseDiv').getText().strip().split(":")
    
    #fetching and returning the prices we are interested in
    for item in data_array:
        if 'lastPrice' in item:   #last traded price
            index = data_array.index(item)+1
            latestPrice=data_array[index].split('"')[1]
            lp =  float(latestPrice.replace(',',''))
        elif 'open' in item:     #day's opening price
            index = data_array.index(item)+1
            openPrice=data_array[index].split('"')[1]
            op =  float(openPrice.replace(',',''))
        elif 'dayLow' in item:   #lowest price of the day
            index = data_array.index(item)+1
            dayLow=data_array[index].split('"')[1]
            dl =  float(dayLow.replace(',',''))
        elif 'dayHigh' in item:  #highest price of the day
            index = data_array.index(item)+1
            dayHigh=data_array[index].split('"')[1]
            dh =  float(dayHigh.replace(',',''))
    return op,lp,dh,dl     #returning the prices 



In [3]:
#url where list of nifty50 stocks get updated incase there is a change in future
nifty50_list = 'https://archives.nseindia.com/content/indices/ind_nifty50list.csv'
#accessing the csv file with the list of stocks using pandas
df_n50 = pd.read_csv(nifty50_list)

In [4]:
#creating empty lists to store the fetched prices
OP  = []
LP  = []
DHP = []
DLP = []
regexp = re.compile('&')  #to search "&" in the column 'symbol' of nifty50_list 
minutes=int(input("enter no. of minutes : "))
for i in range(0,minutes):
   try:
      for index, row in df_n50.iterrows():  #replacing '&' with '%26' to avoid error
         stock_code = row['Symbol']
         if(regexp.search(stock_code) != None):
            stock_code = stock_code.replace('&','%26')
         #calling the function and storing the prices in the respective lists
         oPrice,lPrice,dhPrice, dlPrice = fetch_stock_info(stock_code)  
         OP.append(str(oPrice))
         LP.append(str(lPrice))
         DHP.append(str(dhPrice))
         DLP.append(str(dlPrice))
      sleep(10)
      f=open("nifty50{}.csv".format(i),"a+")
      #printing heading of the table
      #print("---------------------------------------------------------------------------------------------------------------------------------")
      a="{:50s} , {:20s} , {:10s} , {:10s} , {:10s} , {:10s} , \n".format( 'Company Name','Symbol','openPrice','lastPrice','dayHigh','dayLow')
      f.write(a)
      #print("---------------------------------------------------------------------------------------------------------------------------------")

      #printing the prices in the form of a table
      for index, row in df_n50.iterrows():
         stock_code = row['Symbol']
         
         b="{:50s} , {:20s} , {:10s} ,{:10s} , {:10s} , {:10s} , \n".format(str(row['Company Name']), row['Symbol'], OP[index].rjust(10), LP[index].rjust(10), DHP[index].rjust(10), DLP[index].rjust(10))
         f.write(b)
      #print("---------------------------------------------------------------------------------------------------------------------------------")
      

   except KeyboardInterrupt:
        sys.exit()
   finally:
    f.close()

enter no. of minutes : 2
